# Inspect `.safetensors` (MobileNetV2 / TIMM) 🔎

Notebook ini membantu kamu melihat **daftar kunci (keys)**, **shape**, **dtype**, dan membuat **proposed mapping** dari skema TIMM → skema Rust (base/head). Jalankan cell dari atas ke bawah di mesin kamu.

**Catatan:** Notebook ini hanya *membaca* file; tidak mengubah apapun kecuali kalau kamu menjalankan cell untuk menulis CSV mapping.


In [1]:

# === Path kandidat file .safetensors ===
# Ubah PATH di sini kalau perlu
from pathlib import Path

CANDIDATES = [
    "./mobilenet_v2_1_0_imagenet.safetensors",
    "./model.safetensors",
    "./mobilenet_v2_1_0_imagenet.safetensors",
    "./model.safetensors",
]

PATH = None
for c in CANDIDATES:
    if Path(c).exists():
        PATH = c
        break

print("Detected file:", PATH)
if PATH is None:
    print("⚠ File tidak ditemukan di kandidat paths. Set PATH manual.")


Detected file: ./mobilenet_v2_1_0_imagenet.safetensors


In [2]:

# === Import yang dibutuhkan ===
# Prefer numpy loader agar tidak perlu PyTorch untuk sekadar inspeksi.
try:
    from safetensors.numpy import load_file as load_np
except Exception as e:
    raise SystemExit(
        "Tidak bisa import safetensors.numpy. Install dulu:\n"
        "  pip install safetensors\n"
        f"Detail error: {e}"
    )

import pandas as pd
import re, math
from collections import defaultdict


In [3]:

# === Baca file & list keys ===
if PATH is None:
    raise SystemExit("Set PATH ke file .safetensors yang benar lalu jalankan lagi.")

sd = load_np(PATH)   # dict-like: key -> numpy array
print(f"Loaded tensors: {len(sd)}")

# Buat DataFrame index keys
rows = []
total_params = 0
for k, arr in sd.items():
    shape = tuple(arr.shape)
    dtype = str(arr.dtype)
    numel = int(arr.size)
    total_params += numel
    rows.append({"key": k, "shape": shape, "dtype": dtype, "numel": numel})

df = pd.DataFrame(rows).sort_values("key").reset_index(drop=True)
print("Total parameters (numel):", total_params)
df.head(20)


Loaded tensors: 314
Total parameters (numel): 3539036


,key,shape,dtype,numel
0,blocks.0.0.bn1.bias,"(32,)",float32,32
1,blocks.0.0.bn1.num_batches_tracked,(),int64,1
2,blocks.0.0.bn1.running_mean,"(32,)",float32,32
3,blocks.0.0.bn1.running_var,"(32,)",float32,32
4,blocks.0.0.bn1.weight,"(32,)",float32,32
5,blocks.0.0.bn2.bias,"(16,)",float32,16
6,blocks.0.0.bn2.num_batches_tracked,(),int64,1
7,blocks.0.0.bn2.running_mean,"(16,)",float32,16
8,blocks.0.0.bn2.running_var,"(16,)",float32,16
9,blocks.0.0.bn2.weight,"(16,)",float32,16


In [4]:

# === Simpan indeks keys ke CSV (opsional) ===
out_csv = Path(PATH).with_suffix("").name + "_INDEX.csv"
df.to_csv(out_csv, index=False)
print("Saved index to:", out_csv)


Saved index to: mobilenet_v2_1_0_imagenet_INDEX.csv


In [5]:

# === Lihat prefix unik untuk memahami skema penamaan (TIMM biasanya: conv_stem, bn1, blocks.X.conv.*, conv_head, bn2, classifier) ===
def first_tokens(s, n=2):
    toks = s.split(".")
    return ".".join(toks[:n]) if len(toks) >= n else s

df["prefix2"] = df["key"].map(lambda s: first_tokens(s, 2))
df["prefix3"] = df["key"].map(lambda s: first_tokens(s, 3))

print("Prefix level-2:")
display(df["prefix2"].value_counts().to_frame("count"))

print("\nPrefix level-3 (berguna untuk melihat 'blocks.N.conv'):")
display(df["prefix3"].value_counts().to_frame("count").head(30))


Prefix level-2:


,count
prefix2,
blocks.3,72
blocks.2,54
blocks.4,54
blocks.5,54
blocks.1,36
blocks.6,18
blocks.0,12
bn2.running_mean,1
conv_head.weight,1



Prefix level-3 (berguna untuk melihat 'blocks.N.conv'):


,count
prefix3,
blocks.5.2,18
blocks.6.0,18
blocks.1.1,18
blocks.2.0,18
blocks.2.1,18
blocks.2.2,18
blocks.3.0,18
blocks.3.1,18
blocks.3.2,18


In [6]:

# === Helper: cari keys dengan pola (regex / contains) ===
def search(pattern, regex=True):
    if regex:
        pat = re.compile(pattern)
        out = df[df["key"].str.contains(pat)]
    else:
        out = df[df["key"].str.contains(pattern, regex=False)]
    return out.sort_values("key").reset_index(drop=True)

# Contoh:
# search(r"blocks\.4\.conv")
# search("dw_bn", regex=False)


## Proposed mapping: TIMM → Rust naming
Bagian ini **mengusulkan** mapping nama ke skema Rust:

- `conv_stem.weight` → `base.stem.weight`
- `bn1.*` → `base.stem.bn.*`
- `blocks.{k}.conv.pw.*` → `base.ir_i_j.expand.*`
- `blocks.{k}.conv.dw.*` → `base.ir_i_j.dw.*`
- `blocks.{k}.conv.dw_bn.*` → `base.ir_i_j.dw.bn.*`
- `blocks.{k}.conv.pw_linear.*` → `base.ir_i_j.project.*`
- `blocks.{k}.conv.pw_linear_bn.*` → `base.ir_i_j.project.bn.*`
- `conv_head.weight` → `base.last.weight`
- `bn2.*` → `base.last.bn.*`
- (Classifier diabaikan karena head kamu dibuat ulang sesuai num_classes)

**Catatan penting:** index `(i,j)` untuk `ir_i_j` bergantung pada konfigurasi `cfg` (jumlah repeat per stage). Di bawah ini kita hitung `(i,j)` dari indeks blok `k` secara deterministik agar konsisten dengan model Rust kamu.


In [7]:

# === Build proposed mapping table ===
# cfg: (t, c, n, s) — sama dengan di Rust
cfg = [
    (1, 16, 1, 1),
    (6, 24, 2, 2),
    (6, 32, 3, 2),
    (6, 64, 4, 2),
    (6, 96, 3, 1),
    (6,160, 3, 2),
    (6,320, 1, 1),
]

def k_to_ij(k, cfg):
    kk = int(k)
    for i, (_t,_c,n,_s) in enumerate(cfg):
        if kk < n:
            return i, kk
        kk -= n
    raise ValueError(f"k out of range: {k}")

rows_map = []
for k, arr in sd.items():
    dst = None
    if k == "conv_stem.weight":
        dst = "base.stem.weight"
    elif k.startswith("bn1."):
        dst = "base.stem.bn." + k.split(".", 1)[1]
    elif k.startswith("conv_head."):
        dst = "base.last." + k.split(".", 1)[1]
    elif k.startswith("bn2."):
        dst = "base.last.bn." + k.split(".", 1)[1]
    elif k.startswith("blocks."):
        # ex: blocks.4.conv.dw_bn.weight
        parts = k.split(".")
        # expected: ["blocks", "<k>", "conv", "<piece>", "<maybe more>"]
        if len(parts) >= 4 and parts[2] == "conv":
            kk = int(parts[1])
            i, j = k_to_ij(kk, cfg)
            piece = parts[3]  # pw / dw / pw_linear / dw_bn / pw_bn / pw_linear_bn
            tail = ".".join(parts[4:]) if len(parts) > 4 else ""

            if piece == "pw":
                dst = f"base.ir_{i}_{j}.expand.{tail or 'weight'}"
            elif piece == "pw_bn":
                dst = f"base.ir_{i}_{j}.expand.bn.{tail}"
            elif piece == "dw":
                dst = f"base.ir_{i}_{j}.dw.{tail or 'weight'}"
            elif piece == "dw_bn":
                dst = f"base.ir_{i}_{j}.dw.bn.{tail}"
            elif piece == "pw_linear":
                dst = f"base.ir_{i}_{j}.project.{tail or 'weight'}"
            elif piece == "pw_linear_bn":
                dst = f"base.ir_{i}_{j}.project.bn.{tail}"
            else:
                dst = None
        else:
            dst = None

    rows_map.append({"src_key": k, "dst_key": dst})

map_df = pd.DataFrame(rows_map)
display(map_df.head(20))

# Simpan untuk dicek manual
out_csv = "proposed_mapping_TIMM_to_RUST.csv"
map_df.to_csv(out_csv, index=False)
print("Saved mapping proposal to:", out_csv)


,src_key,dst_key
0,blocks.0.0.bn1.bias,None
1,blocks.0.0.bn1.num_batches_tracked,None
2,blocks.0.0.bn1.running_mean,None
3,blocks.0.0.bn1.running_var,None
4,blocks.0.0.bn1.weight,None
5,blocks.0.0.bn2.bias,None
6,blocks.0.0.bn2.num_batches_tracked,None
7,blocks.0.0.bn2.running_mean,None
8,blocks.0.0.bn2.running_var,None
9,blocks.0.0.bn2.weight,None


Saved mapping proposal to: proposed_mapping_TIMM_to_RUST.csv


### Catatan tentang `dw_bn.bias`
Jika muncul error seperti `cannot find the tensor named blocks.4.conv.dw_bn.bias`, itu berarti file yang kamu unduh **tidak memiliki** parameter bias untuk batchnorm tersebut, atau penamaannya berbeda (mis. `bn.weight`, `bn.bias`, `bn.running_mean`, `bn.running_var`).

Cek dengan `search('dw_bn', regex=False)` pada sel di atas untuk melihat kunci apa saja yang benar–benar ada. Kalau `bias` tidak ada, loader di Rust **tidak perlu** mengharapkannya — cukup copy yang ada saja.